# Leetcode 100-199
See https://leetcode.com/problems/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import math
from typing import *

In [ ]:
# utility functions and version check

def run_sample(obj, sample, funcname):
    lines=[s for s in sample.splitlines() if len(s)>0 ]
    data=[json.loads(s) for s in lines]
    func=getattr(obj, funcname)
    res=func(*data[:-1])
    if res!=data[-1]:
        raise ValueError(f'expected {data[-1]}, actual {res}')

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # leetcode nnn)

In [ ]:
# leetcode 122
# https://leetcode.com/problems/best-time-to-buy-and-sell-stock-ii/

sample1='''
[7,1,5,3,6,4]
7
'''

sample2='''
[1,2,3,4,5]
4
'''

sample3='''
[7,6,4,3,1]
0
'''

class Solution:
    # idea: if going up buy, if going down sell
    def maxProfit(self, prices: List[int]) -> int:
        shares=0
        lastbuyprice=None
        profit=0
        prevprice=None
        for price in prices:
            if prevprice is not None:
                if price>prevprice and shares==0:
                    shares=1
                    lastbuyprice=prevprice
                elif price<prevprice and shares==1:
                    shares=0
                    profit+=prevprice-lastbuyprice
                    lastbuyprice=None
            prevprice=price
        # sell what remains at the end (i.e. on the last day)
        if shares==1 and prevprice>lastbuyprice:
            profit+=prevprice-lastbuyprice
        return profit

run_sample(Solution(), sample1, 'maxProfit')
run_sample(Solution(), sample2, 'maxProfit')
run_sample(Solution(), sample3, 'maxProfit')


In [ ]:
# leetcode 189
# https://leetcode.com/problems/rotate-array/

sample1='''
[1,2,3,4,5,6,7]
3
[5,6,7,1,2,3,4]
'''

sample2='''
[-1,-100,3,99]
2
[3,99,-1,-100]
'''

sample3='''
[1,2,3,4,5,6]
4
[3,4,5,6,1,2]
'''

sample4='''
[1,2]
0
[1,2]
'''

sample5='''
[1,2,3,4,5,6,7,8,9,10]
4
[7,8,9,10,1,2,3,4,5,6]
'''

sample6='''
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
38
[17,18,19,20,21,22,23,24,25,26,27,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
'''

class Solution:
    def rotate2(self, nums: List[int], k: int) -> List[int]:
        self.rotate(nums, k)
        return nums

    # idea: in-place, pass through values until coming back to original spot
    # (or not, depending on gcd(len(nums), k) )
    def rotate(self, nums: List[int], k: int) -> None:
        k=k%len(nums)
        if k==0:
            return
        gcd=math.gcd(len(nums), k)
        loopsz=(len(nums)+gcd-1)//gcd
        #print(f'{len(nums)=}, {k=}, {gcd=}, {loopsz=}')
        for i0 in range(gcd):
            i=i0
            n=nums[i]
            for x in range(loopsz):
                newindex=(i+k)%len(nums)
                saved=nums[newindex]
                nums[newindex]=n
                n=saved
                i=newindex

run_sample(Solution(), sample1, 'rotate2')
run_sample(Solution(), sample2, 'rotate2')
run_sample(Solution(), sample3, 'rotate2')
run_sample(Solution(), sample4, 'rotate2')
run_sample(Solution(), sample5, 'rotate2')
run_sample(Solution(), sample6, 'rotate2')